# Data preparation

The goal of this notebook is to prepare the data for the project. In particular, we map the book's metadata (especially the summary) that comes from one dataset to the list of user-book ratings.

In [1]:
import pandas as pd

### Extract books complete metadata, including description (books_1.Best_Books_Ever.csv)

In [2]:
books_full_metadata = pd.read_csv('../books_1.Best_Books_Ever.csv')
books_full_metadata.head(1)
len(books_full_metadata)

52478

Keep only boks that are in english

In [3]:
books_full_metadata = books_full_metadata[books_full_metadata["language"] == "English"]
len(books_full_metadata)

42661

Get only the features we are interested in

In [4]:
books_full_metadata = books_full_metadata[["title","series","author","description","genres","pages", "publisher","firstPublishDate","awards","setting","coverImg"]]

Parse titles

In [5]:
books_full_metadata['mod_title'] = books_full_metadata['title'].str.replace("\s+", " ", regex=True) #Remove multiple spaces in a row
books_full_metadata['mod_title'] = books_full_metadata['mod_title'].str.replace("[^\w\s]", "", regex=True).str.lower() #Remove punctuation and change to lower case
books_full_metadata[books_full_metadata["mod_title"] == "mobydick or the whale"]

,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg,mod_title
100,"Moby-Dick or, the Whale",NaN,"Herman Melville, Andrew Delbanco (Introduction...","""It is the horrible texture of a fabric that s...","['Classics', 'Fiction', 'Literature', 'Adventu...",654,Penguin Classics,10/18/51,['Audie Award for Solo Narration - Male (2006)...,"['Nantucket Island, Massachusetts (United Stat...",https://i.gr-assets.com/images/S/compressed.ph...,mobydick or the whale


Remove books that have the same title 

In [6]:
duplicate_book = books_full_metadata[books_full_metadata["mod_title"].isin(books_full_metadata["mod_title"][books_full_metadata["mod_title"].duplicated()])].sort_values("mod_title").index
books_full_metadata = books_full_metadata.drop(duplicate_book)

In [7]:
len(books_full_metadata)

38829

### Extract goodreads books incomplete metadata (books_titles.json)

In [76]:
books_partial_metadata = pd.read_json("../books_titles.json")
books_partial_metadata["book_id"] = books_partial_metadata["book_id"].astype(str)
books_partial_metadata = books_partial_metadata.set_index("book_id")
books_partial_metadata = books_partial_metadata.drop(columns=["title", "ratings", "url", "cover_image"]) #drop unecessary colums
books_partial_metadata.head()

,mod_title
book_id,
1333909,good harbor
7327624,the unschooled wizard sun wolf and starhawk 12
6066819,best friends forever
287140,runic astrology starcraft and timekeeping in t...
287141,the aeneid for boys and girls


Remove empty titles

In [77]:
books_partial_metadata_removed_ids = books_partial_metadata[books_partial_metadata["mod_title"] == " "].index #rget the ids of books with empty titles
books_partial_metadata = books_partial_metadata[books_partial_metadata["mod_title"] != " "] #remove empty titles
len(books_partial_metadata)

1716607

Remove book duplicates and store the new book ids for the duplicates

In [78]:
#Get the list of books with a duplicate title
duplicate_books = books_partial_metadata[books_partial_metadata.duplicated(subset=['mod_title'],keep=False)]
duplicate_books_title_set = set(duplicate_books["mod_title"])
len(duplicate_books)

701831

In [79]:
duplicate_book_new_id = {} #When a book is a duplicate, only keep the id of the fisrt one encountered
new_aggregates_id = {} #Stores the new id associated to the old id of duplicate books
for book_id, row in duplicate_books.iterrows():
  book_title = row["mod_title"]
  # If the book is a duplicate, then drop the book
  if book_title in duplicate_book_new_id:
    new_aggregates_id[book_id] = duplicate_book_new_id[book_title]
    # books_partial_metadata = books_partial_metadata[books_partial_metadata["book_id"] != book_id]
  else:
    duplicate_book_new_id[book_title] = book_id 
    new_aggregates_id[book_id] = book_id

In [80]:
#One unique id is assigned for each group of books with same title
duplicate_book_new_id_df = pd.DataFrame(duplicate_book_new_id.items(), columns=["mod_title", "book_id"])
duplicate_book_new_id_df = duplicate_book_new_id_df.set_index("book_id")
len(duplicate_book_new_id_df)

212896

In [81]:
#Remove books that have a duplicate
books_partial_metadata = books_partial_metadata[~books_partial_metadata['mod_title'].isin(duplicate_books_title_set)]
len(books_partial_metadata)

1014776

In [82]:
#Add back the aggregated group of books that had same titles
books_partial_metadata = pd.concat([books_partial_metadata, duplicate_book_new_id_df])

In [83]:
books_partial_metadata["book_id"] = books_partial_metadata.index
len(books_partial_metadata)

1227672

### Import mapping between book ids in the csv and books_titles.json file (book_id_map.csv)

In [84]:
csv_book_mapping = {}

with open("../book_id_map.csv", "r") as file: #Reading through large file
    next(file) #Skip header
    while (line := file.readline().rstrip()):
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

### Find the intersection between the two book descriptions dataframes

In [85]:
books_intersection_full_partial = pd.merge(books_full_metadata, books_partial_metadata, how ='inner', on =['mod_title'])

In [86]:
books_intersection_full_partial.head(1)

,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg,mod_title,book_id
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,14796360


In [87]:
#Get the list of all book ids for wich we have a description 
intersection_book_id = set(books_intersection_full_partial["book_id"])
len(intersection_book_id)

26764

In [133]:
# Get the list of new book that were duplicates
# Keep only the id of the book we kept
new_aggregates_id = dict((k,v) for k,v in new_aggregates_id.items() if v in intersection_book_id)
len(new_aggregates_id)

122243

### Extract book ratings for books fo which we have an actual descritption

In [134]:
known_book_ratings = []
i=0
with open("../goodreads_interactions.csv", 'r') as file:
    next(file) #Skip header
    while (line := file.readline().rstrip()):
        #Retrieve user, book id and associated rating
        user_id, csv_book_id, _, rating, _ = line.split(",")
        book_id = csv_book_mapping.get(csv_book_id)
        if book_id in intersection_book_id: #If we have the book description 
            known_book_ratings.append([user_id, book_id, rating])
        elif book_id in new_aggregates_id: #If the book was a duplicate and we also have a description
            known_book_ratings.append([user_id, new_aggregates_id[book_id], rating])
        i+=1
        if(i%5000000==0):
            print(f"{round(i/229000000*100,1)}% completed")

2.2% completed
4.4% completed
6.6% completed
8.7% completed
10.9% completed
13.1% completed
15.3% completed
17.5% completed
19.7% completed
21.8% completed
24.0% completed
26.2% completed
28.4% completed
30.6% completed
32.8% completed
34.9% completed
37.1% completed
39.3% completed
41.5% completed
43.7% completed
45.9% completed
48.0% completed
50.2% completed
52.4% completed
54.6% completed
56.8% completed
59.0% completed
61.1% completed
63.3% completed
65.5% completed
67.7% completed
69.9% completed
72.1% completed
74.2% completed
76.4% completed
78.6% completed
80.8% completed
83.0% completed
85.2% completed
87.3% completed
89.5% completed
91.7% completed
93.9% completed
96.1% completed
98.3% completed


In [135]:
len(known_book_ratings)
#63825044
#56572131

56572131

Put the user book-ratings in a dataframe

In [136]:
users_ratings = pd.DataFrame(known_book_ratings, columns=["user_id", "book_id", "rating"])
users_ratings["rating"] = pd.to_numeric(users_ratings["rating"])
users_ratings.head()

,user_id,book_id,rating
0,0,5602347,5
1,0,30,5
2,0,12528798,5
3,0,25026517,4
4,0,835,4


### Export the dataframes to csv

Export the user ratings of books we know the description of

In [138]:
users_ratings.to_csv("./data/user_book_ratings.csv")

Export the list of rated books with there full metadata

In [154]:
#Set the book id as id and drop mod_title
books_intersection_full_partial = books_intersection_full_partial.set_index("book_id") 
books_intersection_full_partial = books_intersection_full_partial.drop(columns=["mod_title"])
books_intersection_full_partial.head(3)

,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg
book_id,,,,,,,,,,,
14796360,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...
7743507,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",There is a door at the end of a silent corrido...,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",870,Scholastic Inc.,06/21/03,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...
23390821,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",324,Harper Perennial Modern Classics,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...


In [155]:
books_intersection_full_partial.to_csv("./data/books_metadata.csv")